<a href="https://colab.research.google.com/github/Jeansds/Curso_Udemy---Deep-Learning-com-Python-de-A-a-Z/blob/master/Classificacao_Gatos_Cachorros_Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.preprocessing import image

In [4]:
!wget -c https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip

# remove existing directories
!rm -r Cat_Dog_data __MACOSX || true
!unzip -qq Cat_Dog_data.zip

data_dir = 'Cat_Dog_data'

--2020-03-03 14:13:53--  https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.38.206
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.38.206|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [12]:
classificador = Sequential()
classificador.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2, 2)))
classificador.add(Dropout(0.25))

classificador.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2, 2)))
classificador.add(Dropout(0.25))

classificador.add(Flatten())

classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.25))

classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.25))

classificador.add(Dense(units = 1, activation = 'sigmoid'))

classificador.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
gerador_treinamento = ImageDataGenerator(rescale = 1./255, rotation_range = 7,
                                         shear_range = 0.2, height_shift_range = 0.07, zoom_range = 0.2)
gerador_teste = ImageDataGenerator(rescale = 1./255)
#o metodo flow_from_directory automaticamente determina a quantidade de classes e o seu tipo pelos respectivos
#nomes e quantidades de pastas no diretorio indicado
base_treinamento = gerador_treinamento.flow_from_directory('/content/Cat_Dog_data/train', 
                                                           target_size = (64, 64), batch_size = 32,
                                                           class_mode = 'binary')
base_teste = gerador_teste.flow_from_directory('/content/Cat_Dog_data/test',
                                                target_size = (64, 64), batch_size = 32,
                                                class_mode = 'binary')

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [15]:
#steps_per_epoch é a quantidade de imagens usadas por epoca, bom sempre deixar o total de imagens da pasta maior + lento
#a mesmma coisa vale para validation steps e epochs, então boa sorte no futuro Jean
classificador.fit_generator(base_treinamento, steps_per_epoch = 22500/32, epochs = 5,
                            validation_data = base_teste, validation_steps = 2500/32)



Epoch 1/5
704/703 [==============================] - 95s 135ms/step - loss: 0.6770 - acc: 0.6108 - val_loss: 0.6058 - val_acc: 0.6752
Epoch 2/5
704/703 [==============================] - 88s 124ms/step - loss: 0.5992 - acc: 0.6762 - val_loss: 0.5644 - val_acc: 0.6964
Epoch 3/5
704/703 [==============================] - 84s 120ms/step - loss: 0.5590 - acc: 0.7068 - val_loss: 0.7298 - val_acc: 0.6404
Epoch 4/5
704/703 [==============================] - 87s 123ms/step - loss: 0.5256 - acc: 0.7365 - val_loss: 0.5441 - val_acc: 0.7292
Epoch 5/5
704/703 [==============================] - 83s 118ms/step - loss: 0.5003 - acc: 0.7539 - val_loss: 0.5270 - val_acc: 0.7332


In [22]:
#teste da rede com uma img
imagem_teste = image.load_img('/content/Cat_Dog_data/test/dog/dog.10010.jpg', target_size = (64,64))
imagem_teste = image.img_to_array(imagem_teste)
imagem_teste /= 255
imagem_teste = np.expand_dims(imagem_teste, axis = 0)# adiciona uma coluna, a img é 64, 64, 3, com isto ela vira 1, 64, 64, 3 pois o tensor flow precisa deste formato para funcionar
print(classificador.predict(imagem_teste))
print(base_treinamento.class_indices)

[[0.7815035]]
{'cat': 0, 'dog': 1}
